In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import pandas as pd
from tqdm import tqdm
import time
pd.options.display.max_rows = None

# array 결과 다 나오게 하기
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

In [2]:
review10 = pd.read_csv("osaka_review1-10.csv")
review10

,name,rating,id,country,title,review,type,translation,votes,date of stay
0,HOTEL THE FLAG Shinsaibashi,5,Lovecathcath,"Hong Kong, China",2分鐘直達心齋橋~最近心齋橋嘅酒店,2分鐘直達心齋橋~最近心齋橋嘅酒店 今次同大家介紹呢間酒店非常之近心齋橋， 酒店名「HOT...,Traveled with family,G,242.0,December 2022
1,HOTEL THE FLAG Shinsaibashi,5,AnthonyChng,"Singapore, Singapore",Second Time in Osaka,"Great stay, everything is new and nice. Bath i...",Traveled with family,NaN,4.0,December 2022
2,HOTEL THE FLAG Shinsaibashi,5,Jacek B,"Katowice, Poland","Amazing hotel, lovely staff",Our stay at this hotel was one of the best we'...,Traveled as a couple,NaN,NaN,November 2022
3,HOTEL THE FLAG Shinsaibashi,5,Jayd822,SA,Highly recommend!,Really enjoyed our stay at this modern funky h...,Traveled with friends,NaN,79.0,February 2020
4,HOTEL THE FLAG Shinsaibashi,5,Harald Gracholski,"Cologne, Germany",Best possible location in Osaka!,The Flag in Shinsaibashi was probably the most...,NaN,NaN,529.0,November 2019
5,HOTEL THE FLAG Shinsaibashi,5,Librilisa,United Kingdom,"Modern, Funky Hotel with Great Amenities - AMA...",This seems like a newer hotel as everything lo...,NaN,NaN,15.0,March 2020
6,HOTEL THE FLAG Shinsaibashi,5,Jason S,NaN,Hard to find better!,"Second time I've stayed here, love it! Great l...",NaN,NaN,3.0,March 2020
7,HOTEL THE FLAG Shinsaibashi,5,J M,"Melbourne, Australia",Such a surprise!,My stay here really exceeded expectations. The...,NaN,NaN,56.0,February 2020
8,HOTEL THE FLAG Shinsaibashi,5,Danielle,"Perth, Australia",Excellent aesthetic,Great service. Very clean rooms which are new....,Traveled as a couple,NaN,2.0,March 2020
9,HOTEL THE FLAG Shinsaibashi,5,Sndr23,"Utrecht Province, The Netherlands",Modern design - ‘larger’ rooms & great location,Have spend quite some trips to Japan over the ...,Traveled on business,NaN,6.0,March 2020


In [18]:
review10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4091 entries, 0 to 4090
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          4091 non-null   object 
 1   rating        4091 non-null   int64  
 2   id            4034 non-null   object 
 3   country       2836 non-null   object 
 4   title         4091 non-null   object 
 5   review        4091 non-null   object 
 6   type          62 non-null     object 
 7   translation   4091 non-null   object 
 8   votes         3507 non-null   float64
 9   date of stay  4091 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 319.7+ KB


In [17]:
review10['name'].unique()

array(['HOTEL THE FLAG Shinsaibashi', 'Conrad Osaka',
       'Hotel Universal Port Vita', 'Fraser Residence Nankai Osaka',
       'InterContinental Hotel Osaka', 'The Bridge Hotel Shinsaibashi',
       'karaksa hotel Osaka Namba', 'Marriott Osaka Miyako Hotel',
       'Imperial Hotel Osaka', 'Cross Hotel Osaka'], dtype=object)

### 호텔 순위 (from tripadvisor) 추가하기

In [27]:
rank = []
for i in review10['name']:
    if i == 'HOTEL THE FLAG Shinsaibashi':
        rank.append(1)
    elif i == 'Conrad Osaka':
        rank.append(2)
    elif i == 'Hotel Universal Port Vita':
        rank.append(3)
    elif i == 'Fraser Residence Nankai Osaka':
        rank.append(4)
    elif i == 'InterContinental Hotel Osaka':
        rank.append(5)
    elif i == 'The Bridge Hotel Shinsaibashi':
        rank.append(6)
    elif i == 'karaksa hotel Osaka Namba':
        rank.append(7)
    elif i == 'Marriott Osaka Miyako Hotel':
        rank.append(8)
    elif i == 'Imperial Hotel Osaka':
        rank.append(9)
    elif i == 'Cross Hotel Osaka':
        rank.append(10)        
    else : 
        rank.append(11)


In [33]:
review10['ranking'] = rank
review10.tail(10)

,name,rating,id,country,title,review,type,translation,votes,date of stay,ranking
4081,Cross Hotel Osaka,5,heitung1215,"Hong Kong, China",超方便的酒店,"酒店位置十分方便,房間不大,但很乾淨,對面就是激安商場,而且鄰近心齋橋,有很多不同食店,行去...",NaN,G,8.0,March 2017,10
4082,Cross Hotel Osaka,5,apolla76,"Milan, Italy",Ottima soluzione,dopo tokyo e kyoto abbiamo soggiornato due not...,NaN,G,5.0,January 2018,10
4083,Cross Hotel Osaka,5,Emily L,NaN,Perfect central location,My boyfriend and I stayed in this hotel in Nov...,NaN,E,34.0,November 2017,10
4084,Cross Hotel Osaka,4,TOKD,"Gwelup, Australia",Comfortable,We stayed for 4 nights. The bed was a bit smal...,NaN,E,44.0,January 2018,10
4085,Cross Hotel Osaka,5,Bordenave F,"Dover, New Hampshire",Great Hotel,I was a participant in the Osaka Marathon and ...,NaN,E,NaN,November 2017,10
4086,Cross Hotel Osaka,5,大大大寶,Taipei,可以從房間直接看到固利果先生,房間窗戶一打開就是跟固利果先生面對面，別人要特地看的景點，從窗戶就可以看到，飯店到心齋橋只需...,NaN,G,26.0,February 2017,10
4087,Cross Hotel Osaka,5,Akemi K,"Mishima, Japan",次の機会もぜひ,今まで、いろいろな大阪のホテルを使ってきましたが、その中でも立地と設備が抜群にいいです。他に...,NaN,G,17.0,December 2017,10
4088,Cross Hotel Osaka,5,LDLDLD,"Hong Kong, China",Very convenient and great service,"Very nice boutique hotel. Excellent location, ...",NaN,E,62.0,December 2017,10
4089,Cross Hotel Osaka,4,angieyca,"London, United Kingdom",Perfect location and spacious room,First time visiting Osaka and we're so pleased...,NaN,E,140.0,December 2017,10
4090,Cross Hotel Osaka,5,Sihwee,"Singapore, Singapore",Excellent experience,My Boyfriend and I stayed at Cross Hotel for 5...,NaN,E,5.0,December 2017,10


### translation 컬럼 결측값 문자 'E'로 채우기

In [5]:
review10['translation']=review10['translation'].fillna('E')

In [6]:
review10.tail(10)

,name,rating,id,country,title,review,type,translation,votes,date of stay
4081,Cross Hotel Osaka,5,heitung1215,"Hong Kong, China",超方便的酒店,"酒店位置十分方便,房間不大,但很乾淨,對面就是激安商場,而且鄰近心齋橋,有很多不同食店,行去...",NaN,G,8.0,March 2017
4082,Cross Hotel Osaka,5,apolla76,"Milan, Italy",Ottima soluzione,dopo tokyo e kyoto abbiamo soggiornato due not...,NaN,G,5.0,January 2018
4083,Cross Hotel Osaka,5,Emily L,NaN,Perfect central location,My boyfriend and I stayed in this hotel in Nov...,NaN,E,34.0,November 2017
4084,Cross Hotel Osaka,4,TOKD,"Gwelup, Australia",Comfortable,We stayed for 4 nights. The bed was a bit smal...,NaN,E,44.0,January 2018
4085,Cross Hotel Osaka,5,Bordenave F,"Dover, New Hampshire",Great Hotel,I was a participant in the Osaka Marathon and ...,NaN,E,NaN,November 2017
4086,Cross Hotel Osaka,5,大大大寶,Taipei,可以從房間直接看到固利果先生,房間窗戶一打開就是跟固利果先生面對面，別人要特地看的景點，從窗戶就可以看到，飯店到心齋橋只需...,NaN,G,26.0,February 2017
4087,Cross Hotel Osaka,5,Akemi K,"Mishima, Japan",次の機会もぜひ,今まで、いろいろな大阪のホテルを使ってきましたが、その中でも立地と設備が抜群にいいです。他に...,NaN,G,17.0,December 2017
4088,Cross Hotel Osaka,5,LDLDLD,"Hong Kong, China",Very convenient and great service,"Very nice boutique hotel. Excellent location, ...",NaN,E,62.0,December 2017
4089,Cross Hotel Osaka,4,angieyca,"London, United Kingdom",Perfect location and spacious room,First time visiting Osaka and we're so pleased...,NaN,E,140.0,December 2017
4090,Cross Hotel Osaka,5,Sihwee,"Singapore, Singapore",Excellent experience,My Boyfriend and I stayed at Cross Hotel for 5...,NaN,E,5.0,December 2017


### country, city 나누기 (country 컬럼)

In [50]:
tail10 = review10[23:33]
tail10

,name,rating,id,country,title,review,type,translation,votes,date of stay,ranking
23,HOTEL THE FLAG Shinsaibashi,5,Gracia83,"Jakarta, Indonesia",Very Good Stay,I was pleasantly surprised at how good a stay ...,NaN,E,43.0,December 2019,1
24,HOTEL THE FLAG Shinsaibashi,5,David T,"Auckland, New Zealand",Flag flying,Really enjoyed our 2 night stay at the flag ho...,NaN,E,74.0,January 2020,1
25,HOTEL THE FLAG Shinsaibashi,5,Dave,NaN,Really enjoyable stay,"Beautiful modern hotel, loved the shower room ...",NaN,E,1.0,January 2020,1
26,HOTEL THE FLAG Shinsaibashi,5,Onechip Oneputt,"Toronto, Canada",Nice,"The location is prime, very conveniently locat...",NaN,E,5.0,January 2020,1
27,HOTEL THE FLAG Shinsaibashi,5,Ken K,"Sydney, New South Wales, Australia",Top notch,This hotel was an absolute delight to stay in....,NaN,E,50.0,January 2020,1
28,HOTEL THE FLAG Shinsaibashi,4,kosbyfu,"Hong Kong, China",設計時尚簡約 地點方便,當初選他因為設計很好看，住下來覺得比其他日本酒店面積算大。2/F有免費咖啡，有個librar...,NaN,G,29.0,January 2020,1
29,HOTEL THE FLAG Shinsaibashi,5,LT Pan,"Romford, United Kingdom",100% love it,"Fantastic stay. A really great hotel. Clean, s...",NaN,E,1.0,January 2020,1
30,HOTEL THE FLAG Shinsaibashi,5,ReneSiegel,"Livermore, California","Stunning hotel, ideal location",We could not have had a better experience at t...,NaN,E,4.0,December 2019,1
31,HOTEL THE FLAG Shinsaibashi,5,Meagan S,"Edmonton, Canada",Hipster with a side of Zen,"A great centrally located hotel, The Flag live...",NaN,E,55.0,January 2020,1
32,HOTEL THE FLAG Shinsaibashi,5,Bulkin,"Florence, Italy",Ottimo,Ottima struttura moderna (non facile da trovar...,NaN,G,156.0,June 2019,1


### 영어 리뷰가 몇개인지 확인하기

In [8]:
eng_re = review10[review10['translation']=='E']
len(eng_re)

2183

In [16]:
review10['country'].value_counts()

Hong Kong, China                                 214
Singapore, Singapore                             170
Osaka, Japan                                      89
Tokyo, Japan                                      74
Sydney, Australia                                 70
London, United Kingdom                            58
Melbourne, Australia                              54
Taipei, Taiwan                                    44
Bangkok, Thailand                                 29
Japan                                             29
New York City, New York                           29
Tokyo Prefecture, Japan                           27
San Francisco, California                         26
Paris, France                                     26
Seoul, South Korea                                25
Singapore                                         25
Los Angeles, California                           23
Nagoya, Japan                                     22
Brisbane, Australia                           

In [ ]:
# Top 10 호텔을 방문한 고객들의 중요하다고 생각하는 부분은 무엇인가?